In [1]:
# Visualization for prediction cell-type composition using Spatial-DC and eight state-of-the-art deconvolution methods
# Yuan
# 20241119

In [3]:
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib as mat
import os
import sys
from scipy import stats

import cell2location
import matplotlib.colors as mcolors

import warnings
from sklearn.preprocessing import MinMaxScaler,StandardScaler
warnings.filterwarnings("ignore")

plt.rcParams['font.size'] = 10
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42



Global seed set to 0
2024-11-20 14:49:10.449582: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Visualization of deconvolution results without noise
- Fig. 2c and 2d

In [18]:
# visualization of spatial maps without noise and windows size 10
os.chdir("/aaa/zihanwu/yyyli2/project1_spatial_deconv/Manuscript_V20241119/Ablation_study/NSCLC_cond3")

scaler = MinMaxScaler(feature_range=(0, 1))

selected_cells = ["C10","RAW","SVEC"]
methods = ["GroundTruth","SpatialDC","Tangram","Cell2location","Stereoscope","DestVI","Seurat","SPOTlight","SpatialDWLS","CARD"]
sp_adata = sc.read_h5ad("01_data/simulations/synthetic_cellnum_noise_v4/Simu_seed0_cells10_noise0.h5ad")

for i, m in enumerate(methods):
    print(m)
    if (m == "SpatialDC"):
        pred_df = pd.read_csv(f"03_output/exp_conditions_v4/{m}/{m}_reference_noise0_seed0_cells10_noise0_reconstruct.csv",index_col=0)
    elif (m == "GroundTruth"):
        pred_df = pd.read_csv(f"01_data/simulations/synthetic_cellnum_noise_v4/Simu_seed0_cells10_noise0_gd.csv",index_col=0)
    else:
        pred_df = pd.read_csv(f"03_output/exp_conditions_v4/{m}/{m}_reference_noise0_seed0_cells10_noise0.csv",index_col=0)
    
    sp_adata.obs = pred_df[selected_cells]

    pred_df = pd.DataFrame(scaler.fit_transform(sp_adata.obs), columns=sp_adata.obs.columns, index=sp_adata.obs.index)
    sp_adata.obs = pred_df

    fig = cell2location.plt.plot_spatial(sp_adata, show_img=False, labels=selected_cells,
                                          color=selected_cells, circle_diameter=4.5, reorder_cmap=[1,3,2], max_color_quantile=1)

    fig.savefig(f"04_analysis/vis_benchmark/windows10_noise0_{m}_spatial_map.pdf")
    plt.close()
    # plt.show()

GroundTruth
SpatialDC
Tangram
Cell2location
Stereoscope
DestVI
Seurat
SPOTlight
SpatialDWLS
CARD


In [60]:
# Fig.2d-f and Fig.S2e, Bar plots of benchmark metrics, involving CCC, RMSE, PCC and RMSE
os.chdir("/aaa/zihanwu/yyyli2/project1_spatial_deconv/Manuscript_V20241119/Ablation_study/NSCLC_cond3")

method_list = ["SpatialDC_reconstruct","Tangram","Cell2location","Stereoscope","DestVI","Seurat","SPOTlight","SpatialDWLS","CARD"]
method_for_label = ["Spatial-DC","Tangram","cell2location","Stereoscope","DestVI","Seurat","SPOTlight","SpatialDWLS","CARD"] # rename
celltype_orders = ["C10","RAW","SVEC"]
eva_metrics = ["CCC","RMSE","PCC","SPCC"]

benchmark_df = pd.read_csv("04_analysis/cond3_v4_benchmark_cellperc_20241115.csv", index_col=0)  
benchmark_df = benchmark_df[benchmark_df["Seed"] == 0]
benchmark_df = benchmark_df[benchmark_df["Method"].isin(method_list)]
benchmark_df = benchmark_df[benchmark_df["Ref_Noise"] == 0]
benchmark_df = benchmark_df[benchmark_df["Cells"] == 10]
benchmark_df = benchmark_df[benchmark_df["Noise"] == 0]
benchmark_df = benchmark_df[benchmark_df["Cell Type"].isin(celltype_orders)]
# benchmark_df.to_csv("04_analysis/vis_benchmark/eva_windows10_noise0_spatial_bar_add_num.csv") # save to local

eva_mean_std_df = benchmark_df.groupby("Method")[eva_metrics].agg(['mean', 'std']).reset_index()  
cols = ['Method']
for col in eva_metrics:
    for eva in ['mean','std']:
        cols.append(f'{eva}_{col}')

eva_mean_std_df.columns = cols
 
# reset index and reorder  
eva_mean_std_df = eva_mean_std_df.set_index('Method').reindex(method_list)  
  
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8, 6), sharey=False)  
  
for i, metric in enumerate(eva_metrics):  
    mean_val = eva_mean_std_df[f'mean_{metric}']  
    std_val = eva_mean_std_df[f'std_{metric}']  
  
    x = np.arange(len(method_list))  
    y = round(mean_val, 3)  
    yerr = round(std_val, 3)  
      
    # axs[i//2, i%2].bar(x, y, yerr=yerr, width=0.7, capsize=2, linewidth=0)  
    axs[i//2, i%2].bar(x, y, width=0.7, capsize=0, linewidth=0)  
    for j in range(len(x)):  
        axs[i//2, i%2].text(x=j, y=y[j]+0.01*(1 if max(y) <= 1 else max(y)), s=f'{y[j]:.3f}', ha='center', va='bottom')  
  
    axs[i//2, i%2].set_title(metric)  
    axs[i//2, i%2].set_xticks(x)  
    axs[i//2, i%2].set_xticklabels(method_for_label, rotation=90)  
    axs[i//2, i%2].set_ylim(top=1)
    
plt.tight_layout()  
# plt.show()
fig.savefig(f"04_analysis/vis_benchmark/eva_windows10_noise0_spatial_bar_add_num.pdf")

# Robust experiment of Spatial-DC and other start-of-the-art methods on NSCLC dataset under varying Gaussian noise and variable proteins
- Fig. 2g,h; Fig. S2f,g

In [34]:
# run R script to generate the data